In [1]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-2x2x2/"

In [2]:
def print_2d_matrix(matrix, decimals=3):
    for row in matrix:
        print(" ".join(f"{val:9.{decimals}f}" for val in row))

In [3]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [4]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# WFN data
wfndata = wfn2py(dirname+'WFN.h5')
wfnqdata = wfn2py(dirname+'WFNq.h5')

# RHO data
rho = inp.read_rho(dirname+"RHO")

# Vxc data
vxc = inp.read_vxc(dirname+"vxc.dat")

In [5]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 12856.10it/s]


In [6]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv
        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))

        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 8/8 [00:00<00:00, 143.43it/s]

Standard deviation exceeded 1e-16 tolerance: 2.5494915419483335e-15, for i_q:0


In [7]:
from qtm.gw.sigma import Sigma

sigma = Sigma.from_data(
    wfndata=wfndata,
    wfnqdata=wfnqdata,
    sigmainp=sigmainp,
    epsinp=epsinp,
    l_epsmats=epsilon.l_epsinv,
    rho=rho,
    vxc=vxc,
)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 18226.20it/s]


vcoul: Vcoul:
        * gspace = <qtm.gspace.gspc.GSpace object at 0x759279891490>
        * qpts = <qtm.gw.core.QPoints object at 0x7592786b3bd0>
        * bare_coulomb_cutoff = 2.0
        * avgcut = 1e-05
        * l_gspace_q = <class 'list'> of length 8
        * vcoul = <class 'list'> of length 8
        * N_SAMPLES = 2500000.0
        * N_SAMPLES_COARSE = 250000.0
        * SEED = 5000
        


Vcoul calculation for qpts: 100%|██████████| 8/8 [00:05<00:00,  1.49it/s]


In [8]:
from kernel import KernelMtxEl

q0 = [0.001, 0.001, 0.001]
l_qpts = np.array(epsinp.qpts)
l_qpts[0] *= 0
qpts = QPoints.from_cryst(wfndata.kpts.recilat, None, *l_qpts)

kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    q0=q0,
    l_epsmats=epsilon.l_epsinv,
    parallel=True,
)

In [9]:
data = kernelclass.kernel_mtxel()
exc = data["exc"]
head = data["head"]
body = data["body"]
wings = data["wings"]

In [10]:
numq = kernelclass.qpts.numq
numk = kernelclass.kpts.numk

In [11]:
ikf, ikc = np.random.randint(0, numk, size=2)
print(f"ikf, ikc = {ikf}, {ikc}")

ikf, ikc = 0, 1


In [12]:
from intkernel import InterpMtxEl

InterpClass = InterpMtxEl.from_BGW(
    wfn_finedata=wfndata,
    wfn_coarsedata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    kernel=kernelclass,
)

In [13]:
coeff_val = InterpClass.coeff_mtxel(ikf, ikf, "val")
coeff_con = InterpClass.coeff_mtxel(ikf, ikc, "con")


print(f"coeff_val is")
print_2d_matrix(coeff_val, decimals=3)

print(f"\n coeff_con is")
print_2d_matrix(coeff_con, decimals=3)

coeff_val is
1.000-0.000j -0.000+0.000j -0.000+0.000j 0.000-0.000j
-0.000-0.000j 1.000+0.000j 0.000+0.000j -0.000+0.000j
-0.000-0.000j 0.000-0.000j 1.000+0.000j -0.000+0.000j
0.000+0.000j -0.000-0.000j -0.000+0.000j 1.000-0.000j

 coeff_con is
-0.347+0.264j 0.760+0.405j 0.192+0.123j -0.049-0.119j
-0.168+0.593j -0.694+0.108j 0.305-0.023j -0.157-0.093j
0.168-0.179j -0.008-0.003j 0.889+0.380j 0.039+0.061j
-0.306-0.707j 0.000-0.000j -0.001+0.000j 0.630-0.096j


In [14]:
randomarr = np.random.rand(numk, 8)
print(f"\n randomarr is")
print_2d_matrix(randomarr, decimals=3)



 randomarr is
    0.732     0.244     0.802     0.401     0.555     0.570     0.944     0.397
    0.291     0.807     0.230     0.947     0.707     0.643     0.627     0.883
    0.502     0.963     0.454     0.399     0.752     0.051     0.559     0.787
    0.489     0.755     0.690     0.809     0.578     0.885     0.487     0.662
    0.814     0.238     0.827     0.944     0.704     0.262     0.437     0.486
    0.342     0.768     0.056     0.792     0.818     0.008     0.539     0.784
    0.711     0.686     0.707     0.318     0.895     0.499     0.430     0.245
    0.271     0.455     0.055     0.021     0.819     0.566     0.757     0.887


In [15]:
energyval = InterpClass.interp_energy(randomarr, "val")
energycon = InterpClass.interp_energy(randomarr, "con")


print(f"\n energyval is")
print_2d_matrix(energyval, decimals=3)

print(f"\n energycon is")
print_2d_matrix(energycon, decimals=3)


 energyval is
    0.401     0.802     0.244     0.732
    0.947     0.230     0.807     0.291
    0.399     0.454     0.963     0.502
    0.809     0.690     0.755     0.489
    0.944     0.827     0.238     0.814
    0.792     0.056     0.768     0.342
    0.318     0.707     0.686     0.711
    0.021     0.055     0.455     0.271

 energycon is
    0.555     0.570     0.944     0.397
    0.707     0.643     0.627     0.883
    0.752     0.051     0.559     0.787
    0.578     0.885     0.487     0.662
    0.704     0.262     0.437     0.486
    0.818     0.008     0.539     0.784
    0.895     0.499     0.430     0.245
    0.819     0.566     0.757     0.887


In [16]:
is_equal_val = np.allclose(np.flip(randomarr[:, :4], axis = -1), energyval)
is_equal_con = np.allclose(randomarr[:, 4:], energycon)

print(f"Is the interpolated valence energy equal to the random array? {is_equal_val}")
print(f"Is the interpolated conduction energy equal to the random array? {is_equal_con}")

Is the interpolated valence energy equal to the random array? True
Is the interpolated conduction energy equal to the random array? True


In [17]:
fine_kernel = InterpClass.interp_kernel(head, wings, body, exc, sigma.vcoul)
fine_kernel = fine_kernel[:, :, 0, 0, 0, 0]
print(f"\n fine_kernel is")

print_2d_matrix(fine_kernel, decimals=3)


 fine_kernel is
0.040-0.000j -0.024+0.093j -0.053+0.018j -0.030-0.054j -0.089-0.136j 0.007+0.024j 0.010-0.040j 0.002+0.000j
-0.024-0.093j 0.102+0.000j 0.027+0.041j -0.148+0.142j 0.032-0.035j -0.172+0.104j 0.030-0.000j -0.001-0.008j
-0.053-0.018j 0.027-0.041j 0.102-0.000j 0.037+0.200j -0.012-0.047j -0.001+0.030j 0.110-0.170j 0.000+0.000j
-0.030+0.054j -0.148-0.142j 0.037-0.200j 0.081+0.000j -0.029+0.002j 0.013+0.003j -0.010-0.009j -0.012+0.017j
-0.089+0.136j 0.032+0.035j -0.012+0.047j -0.029-0.002j 0.102-0.000j -0.192-0.058j 0.139+0.151j -0.005+0.006j
0.007-0.024j -0.172-0.104j -0.001-0.030j 0.013-0.003j -0.192+0.058j 0.081+0.000j -0.012-0.007j -0.000+0.001j
0.010+0.040j 0.030+0.000j 0.110+0.170j -0.010+0.009j 0.139-0.151j -0.012+0.007j 0.081-0.000j 0.003+0.021j
0.002-0.000j -0.001+0.008j 0.000-0.000j -0.012-0.017j -0.005-0.006j -0.000-0.001j 0.003-0.021j 0.102-0.000j
